In [ ]:
# =========================
# Phase 2 (Clean) — Feature Engineering + Save as CSV (Store NOT scaled)
# =========================
import os, json
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

OUT_DIR = "artifacts/phase2"
os.makedirs(OUT_DIR, exist_ok=True)

TRAIN_PATH = "train.csv"
STORE_PATH = "store.csv"

TRAIN_OUT = os.path.join(OUT_DIR, "train_features.csv")
VAL_OUT   = os.path.join(OUT_DIR, "val_features.csv")
TEST_OUT  = os.path.join(OUT_DIR, "test_features.csv")

FEATURE_COLS_OUT = os.path.join(OUT_DIR, "feature_cols.json")
SPLIT_META_OUT   = os.path.join(OUT_DIR, "split_meta.json")
SCALER_OUT       = os.path.join(OUT_DIR, "scaler.joblib")

# ----------------------------
# 1) Load + Merge
# ----------------------------
train = pd.read_csv(TRAIN_PATH, low_memory=False) 
store = pd.read_csv(STORE_PATH)

train["Date"] = pd.to_datetime(train["Date"])
if "StateHoliday" in train.columns:
    train["StateHoliday"] = train["StateHoliday"].astype(str).str.replace(r"\.0$", "", regex=True)

if "Open" in train.columns:
    train["Open"] = train["Open"].fillna(1)

df = train.merge(store, on="Store", how="left").sort_values(["Store","Date"]).reset_index(drop=True)
df["Store"] = df["Store"].astype(int)

# ----------------------------
# 2) Feature Functions
# ----------------------------
def add_time_features(df):
    df = df.copy()
    d = df["Date"]
    df["year"] = d.dt.year
    df["month"] = d.dt.month
    df["day"] = d.dt.day
    df["weekofyear"] = d.dt.isocalendar().week.astype(int)
    df["dayofyear"] = d.dt.dayofyear
    df["is_weekend"] = ((d.dt.dayofweek + 1) >= 6).astype(int)
    if "DayOfWeek" in df.columns:
        df["DayOfWeek"] = df["DayOfWeek"].astype(int)
    df["DayOfWeek_calc"] = d.dt.dayofweek + 1
    df["is_month_start"] = d.dt.is_month_start.astype(int)
    df["is_month_end"] = d.dt.is_month_end.astype(int)
    return df

def add_holiday_distance(df):
   
    df = df.copy()
    sh = df["StateHoliday"].astype(str) if "StateHoliday" in df.columns else "0"
    school = df["SchoolHoliday"].fillna(0).astype(int) if "SchoolHoliday" in df.columns else 0
    df["is_holiday_any"] = (((sh != "0") & (sh != "nan")) | (school == 1)).astype(int)

    holiday_dates = np.sort(df.loc[df["is_holiday_any"] == 1, "Date"].dropna().unique())
    if len(holiday_dates) == 0:
        df["days_to_next_holiday"] = np.nan
        df["days_since_prev_holiday"] = np.nan
        return df

    dates = df["Date"].values.astype("datetime64[D]")
    h = holiday_dates.astype("datetime64[D]")

    idx_next = np.searchsorted(h, dates, side="left")
    idx_prev = idx_next - 1

    next_date = np.where(idx_next < len(h), h[idx_next], np.datetime64("NaT"))
    prev_date = np.where(idx_prev >= 0, h[idx_prev], np.datetime64("NaT"))

    df["days_to_next_holiday"] = (next_date - dates).astype("timedelta64[D]").astype(float)
    df["days_since_prev_holiday"] = (dates - prev_date).astype("timedelta64[D]").astype(float)

    df.loc[idx_next >= len(h), "days_to_next_holiday"] = np.nan
    df.loc[idx_prev < 0, "days_since_prev_holiday"] = np.nan
    return df

def add_fourier_terms(df, period=365.25, order=3):
    df = df.copy()
    t = (df["Date"] - df["Date"].min()).dt.days.astype(float).values
    for k in range(1, order + 1):
        df[f"fourier_sin_{k}"] = np.sin(2 * np.pi * k * t / period)
        df[f"fourier_cos_{k}"] = np.cos(2 * np.pi * k * t / period)
    return df

def add_store_derived(df):
   
    df = df.copy()
    if "CompetitionOpenSinceYear" in df.columns and "CompetitionOpenSinceMonth" in df.columns:
        comp_year = df["CompetitionOpenSinceYear"]
        comp_month = df["CompetitionOpenSinceMonth"]
        comp_date = pd.to_datetime(
            dict(year=comp_year.fillna(1900).astype(int),
                 month=comp_month.fillna(1).astype(int),
                 day=1), errors="coerce")
        df["competition_age_days"] = (df["Date"] - comp_date).dt.days
        df.loc[comp_year.isna() | comp_month.isna(), "competition_age_days"] = np.nan

    if "Promo2" in df.columns and "PromoInterval" in df.columns:
        promo_map = {"Jan":1,"Feb":2,"Mar":3,"Apr":4,"May":5,"Jun":6,"Jul":7,"Aug":8,"Sept":9,"Oct":10,"Nov":11,"Dec":12}
        def interval_to_mask(s):
            if pd.isna(s) or not isinstance(s, str) or not s.strip(): return 0
            mask = 0
            for p in [x.strip() for x in s.split(",")]:
                m = promo_map.get(p)
                if m: mask |= (1 << (m-1))
            return mask

        uniq = df["PromoInterval"].fillna("").unique()
        mask_dict = {u: interval_to_mask(u) for u in uniq}
        masks = df["PromoInterval"].fillna("").map(mask_dict).astype(int)

        month_idx = (df["month"].astype(int) - 1).to_numpy()
        month_bit = np.left_shift(1, month_idx).astype(int)
        masks_np = masks.to_numpy(dtype=int)

        df["promo2_month_ok"] = ((masks_np & month_bit) > 0).astype(int)
        df["promo2_active"] = ((df["Promo2"].fillna(0).astype(int) == 1) & (df["promo2_month_ok"] == 1)).astype(int)

    if "CompetitionDistance" in df.columns:
        df["has_competitor_distance"] = df["CompetitionDistance"].notna().astype(int)
    return df

def add_lag_rolling(df, lags=(1,7,14,28), windows=(7,14,28), minp_ratio=0.5):
    df = df.copy().sort_values(["Store", "Date"])
    g = df.groupby("Store", sort=False)["Sales"]

    for L in lags:
        df[f"Sales_lag_{L}"] = g.shift(L)

    s = g.shift(1)  # no leakage
    for w in windows:
        minp = max(2, int(w * minp_ratio))
        roll = s.groupby(df["Store"]).rolling(window=w, min_periods=minp)
        df[f"Sales_roll_mean_{w}"] = roll.mean().reset_index(level=0, drop=True)
        df[f"Sales_roll_std_{w}"]  = roll.std().reset_index(level=0, drop=True)
    return df

def add_cum_means(df):
    df = df.copy().sort_values(["Store","Date"])
    cs = df.groupby("Store")["Sales"].cumsum()
    cnt = df.groupby("Store").cumcount()
    df["Sales_cum_mean_store"] = (cs - df["Sales"]) / cnt.replace(0, np.nan)
    return df


def encode_categoricals(df):
    
    df = df.copy()
    for c in ["Open","Promo","SchoolHoliday"]:
        if c in df.columns:
            df[c] = df[c].fillna(0)

    cat_cols = [c for c in ["StoreType", "Assortment", "StateHoliday", "PromoInterval"] if c in df.columns]
    for c in cat_cols:
        df[c] = df[c].fillna("NA").astype(str)

    if cat_cols:
        df = pd.get_dummies(df, columns=cat_cols, drop_first=False)
    return df

def time_split(df, test_days=42, val_days=42):
    max_date = df["Date"].max()
    test_start = max_date - pd.Timedelta(days=test_days - 1)
    val_start  = test_start - pd.Timedelta(days=val_days)

    train_df = df[df["Date"] < val_start].copy()
    val_df   = df[(df["Date"] >= val_start) & (df["Date"] < test_start)].copy()
    test_df  = df[df["Date"] >= test_start].copy()

    meta = {
        "max_date": str(max_date.date()),
        "val_start": str(val_start.date()),
        "test_start": str(test_start.date()),
        "train_rows": int(len(train_df)),
        "val_rows": int(len(val_df)),
        "test_rows": int(len(test_df)),
        "test_days": int(test_days),
        "val_days": int(val_days),
    }
    return train_df, val_df, test_df, meta

def impute_missing(train_df, val_df, test_df):
    num_cols = train_df.select_dtypes(include=["number"]).columns.tolist()
    
    for c in num_cols:
        if c in ["Sales", "Store", "Customers"]:
            continue
        if train_df[c].isna().any() or val_df[c].isna().any() or test_df[c].isna().any():
          
            median_val = train_df[c].median()
            train_df[c] = train_df[c].fillna(median_val)
            val_df[c]   = val_df[c].fillna(median_val)
            test_df[c]  = test_df[c].fillna(median_val)
            
    return train_df, val_df, test_df

# ----------------------------
# 3) Build features
# ----------------------------
df2 = df.copy()
df2 = add_time_features(df2)
df2 = add_holiday_distance(df2)
df2 = add_fourier_terms(df2, order=3)
df2 = add_store_derived(df2)

df2 = add_lag_rolling(df2)
df2 = add_cum_means(df2)

if "Promo" in df2.columns and "SchoolHoliday" in df2.columns:
    df2["promo_x_schoolholiday"] = df2["Promo"].astype(int) * df2["SchoolHoliday"].astype(int)

df2 = encode_categoricals(df2)

# ----------------------------
# 4) Split
# ----------------------------
train_df, val_df, test_df, split_meta = time_split(df2, test_days=42, val_days=42)

# ----------------------------
# 5) Handle Missing (Strictly AFTER split)
# ----------------------------

train_df, val_df, test_df = impute_missing(train_df, val_df, test_df)

# ----------------------------
# 6) Define features 
# ----------------------------
TARGET = "Sales"

DROP_COLS = {"Date", "Store", "Customers", "LogSales"}  

feature_cols = [c for c in df2.columns if (c not in DROP_COLS) and (c != TARGET)]

with open(FEATURE_COLS_OUT, "w", encoding="utf-8") as f:
    json.dump({"feature_cols": feature_cols}, f, ensure_ascii=False, indent=2)

with open(SPLIT_META_OUT, "w", encoding="utf-8") as f:
    json.dump(split_meta, f, ensure_ascii=False, indent=2)

# ----------------------------
# 7) Scale only FEATURES (Store untouched)
# ----------------------------
scaler = StandardScaler()
X_train = train_df[feature_cols].values
X_val   = val_df[feature_cols].values
X_test  = test_df[feature_cols].values

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

joblib.dump(scaler, SCALER_OUT)

# ----------------------------
# 8) Save CSV (scaled features + target + Store + Date)
# ----------------------------
def to_feature_df(split_df, X_scaled):
    out = pd.DataFrame(X_scaled, columns=feature_cols)
   
    out["Sales"] = np.log1p(split_df["Sales"].values) 
    out["Store"] = split_df["Store"].astype(int).values
    out["Date"]  = split_df["Date"].dt.strftime("%Y-%m-%d")
    return out

train_feat = to_feature_df(train_df, X_train_scaled)
val_feat   = to_feature_df(val_df,   X_val_scaled)
test_feat  = to_feature_df(test_df,  X_test_scaled)

train_feat.to_csv(TRAIN_OUT, index=False)
val_feat.to_csv(VAL_OUT, index=False)
test_feat.to_csv(TEST_OUT, index=False)

print("Saved CSVs to:", OUT_DIR)
print("Shapes:", train_feat.shape, val_feat.shape, test_feat.shape)
print("Is 'Customers' leaked into features?", "Customers" in feature_cols)

Saved CSVs to: artifacts/phase2
Shapes: (923549, 62) (46830, 62) (46830, 62)
Is 'Customers' leaked into features? False
